# Page 1-3

In [1]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko Chrome/120.0.6350.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6350.90 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/face-toners-cleansers-skin-care-beauty/-/N-69lc8?moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list = []

def scrape_products():
    global product_list, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Face Toner",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面
✅ WebDriver 关闭
📦 共抓取商品（去重后）：70 条


In [2]:
df = pd.DataFrame(product_list)
df

product_name  \
0   Thayers Natural Remedies Witch Hazel Alcohol F...   
1   Thayers Natural Remedies 2% AHA Exfoliating To...   
2   The Ordinary Glycolic Acid 7% Toning Solution ...   
3   CeraVe Hydrating Toner for Face, Alcohol Free ...   
4   Neutrogena Alcohol-Free Gentle Daily Facial To...   
..                                                ...   
65        MEDIHEAL Teatree Trouble Toner Pads - 100ct   
66       MEDIHEAL Collagen Ampoule Toner Pads - 100ct   
67    MEDIHEAL Phyto-enzyme Peeling Toner Pads - 90ct   
68  MEDIHEAL Retinol Collagen Lifting Toner Pads -...   
69  Natural Outcome Vegan Teenskin Everday Face Ki...   

                       brand   current_price           unit_price  \
0   Thayers Natural Remedies  $4.49 - $12.39                  N/A   
1   Thayers Natural Remedies          $16.29                  N/A   
2               The Ordinary  $8.70 - $13.00                  N/A   
3                     CeraVe          $12.49  ($1.84/fluid ounce)   
4                 Neutrogena           $6.59  ($0.82/fluid ounce)   
..                       ...             ...                  ...   
65                  MEDIHEAL          $23.99        ($0.24/count)   
66                  MEDIHEAL          $23.99        ($0.24/count)   
67                  MEDIHEAL          $24.49        ($0.27/count)   
68                  MEDIHEAL          $23.99        ($0.24/count)   
69           Natural Outcome          $41.99                  N/A   

                             star_rating review_counts  is_on_sale  \
0   4.7 out of 5 stars with 4108 ratings  4108 reviews           0   
1    4.8 out of 5 stars with 213 ratings   213 reviews           0   
2   4.4 out of 5 stars with 3342 ratings  3342 reviews           0   
3   4.7 out of 5 stars with 1535 ratings  1535 reviews           0   
4    4.2 out of 5 stars with 424 ratings   424 reviews           0   
..                                   ...           ...         ...   
65    4.8 out of 5 stars with 23 ratings    23 reviews           0   
66    4.8 out of 5 stars with 16 ratings    16 reviews           0   
67    4.2 out of 5 stars with 10 ratings    10 reviews           0   
68     4.9 out of 5 stars with 7 ratings     7 reviews           0   
69                                   N/A     0 reviews           0   

   is_free_shipping  is_3day_delivery product_category  \
0               N/A                 1       Face Toner   
1                 0                 1       Face Toner   
2               N/A                 1       Face Toner   
3                 0                 1       Face Toner   
4                 0                 1       Face Toner   
..              ...               ...              ...   
65                0                 1       Face Toner   
66                0                 1       Face Toner   
67                0                 1       Face Toner   
68                0                 1       Face Toner   
69                1                 0       Face Toner   

                                         product_link  
0   https://www.target.com/p/thayers-natural-remed...  
1   https://www.target.com/p/thayers-natural-remed...  
2   https://www.target.com/p/the-ordinary-glycolic...  
3   https://www.target.com/p/cerave-hydrating-tone...  
4   https://www.target.com/p/neutrogena-alcohol-fr...  
..                                                ...  
65  https://www.target.com/p/mediheal-teatree-trou...  
66  https://www.target.com/p/mediheal-collagen-amp...  
67  https://www.target.com/p/mediheal-phyto-enzyme...  
68  https://www.target.com/p/mediheal-retinol-coll...  
69  https://www.target.com/p/natural-outcome-vegan...  

[70 rows x 11 columns]

In [3]:
df.to_csv("toner_data.csv",index=False)

# Page 4-6

In [4]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko Chrome/120.0.6350.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6350.90 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/face-toners-cleansers-skin-care-beauty/-/N-69lc8?moveTo=product-list-grid&Nao=72"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list2 = []

def scrape_products():
    global product_list2, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list2.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Face Toner",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 4
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list2)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
⚠️ 超时未加载出 25 个商品，当前页可能不完整
🔍 当前页面商品数: 24
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
⚠️ 超时未加载出 25 个商品，当前页可能不完整
🔍 当前页面商品数: 24
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
⚠️ 超时未加载出 25 个商品，当前页可能不完整
🔍 当前页面商品数: 24
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 4 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
⚠️ 超时未加载出 25 个商品，当前页可能不完整
🔍 当前页面商品数: 10
❌ 没有找到下一页按钮，结束爬取
✅ WebDriver 关闭
📦 共抓取商品（去重后）：82 条


In [5]:
df2 = pd.DataFrame(product_list2)
df2

product_name  \
0   Kiehl's Ultra Facial Toner for All Skin Types,...   
1   Kiehl's Cucumber Herbal Alcohol Free Toner, 16...   
2   NYX Stripped Off Micellar Water - 13.5 oz (PAC...   
3   Natural Outcome OilySkin Mattifying Toner (8 o...   
4                BYOMA Brightening Toner - 5.07 fl oz   
..                                                ...   
77  Mydentity Guy Tang Demi-Permanent - My Dentity...   
78  Wella Color Charm Toner - Permanent Creme Tone...   
79  Alfaparf Milano Evolution of the Color Hair Dy...   
80  Schwarzkopf BlondMe Bond Enforcing Blonding To...   
81  Matrix SoColor Sync (6T Dark Titanium Blonde (...   

                      brand current_price unit_price  \
0                   Kiehl's        $22.82        N/A   
1                   Kiehl's        $43.15        N/A   
2   NYX Professional Makeup        $19.99        N/A   
3           Natural Outcome        $16.99        N/A   
4                     BYOMA        $14.99        N/A   
..                      ...           ...        ...   
77               #MYDENTITY        $17.99        N/A   
78                    Wella        $12.99        N/A   
79                 Alfaparf        $15.99        N/A   
80              Schwarzkopf        $16.99        N/A   
81                   Matrix        $19.99        N/A   

                            star_rating review_counts  is_on_sale  \
0                                   N/A     0 reviews           1   
1                                   N/A     0 reviews           1   
2                                   N/A     0 reviews           0   
3                                   N/A     0 reviews           0   
4   4.2 out of 5 stars with 373 ratings   373 reviews           0   
..                                  ...           ...         ...   
77                                  N/A     0 reviews           0   
78                                  N/A     0 reviews           0   
79    2.3 out of 5 stars with 3 ratings     3 reviews           1   
80                                  N/A     0 reviews           0   
81                                  N/A     0 reviews           0   

   is_free_shipping  is_3day_delivery product_category  \
0                 0                 0       Face Toner   
1                 1                 0       Face Toner   
2                 0                 0       Face Toner   
3                 0                 0       Face Toner   
4                 0                 1       Face Toner   
..              ...               ...              ...   
77                0                 0       Face Toner   
78                0                 0       Face Toner   
79                0                 0       Face Toner   
80                0                 0       Face Toner   
81                0                 0       Face Toner   

                                         product_link  
0   https://www.target.com/p/kiehl-s-ultra-facial-...  
1   https://www.target.com/p/kiehl-s-cucumber-herb...  
2   https://www.target.com/p/nyx-stripped-off-mice...  
3   https://www.target.com/p/natural-outcome-oilys...  
4   https://www.target.com/p/byoma-brightening-ton...  
..                                                ...  
77  https://www.target.com/p/mydentity-guy-tang-de...  
78  https://www.target.com/p/wella-color-charm-ton...  
79  https://www.target.com/p/alfaparf-milano-evolu...  
80  https://www.target.com/p/schwarzkopf-blondme-b...  
81  https://www.target.com/p/matrix-socolor-sync-6...  

[82 rows x 11 columns]

In [6]:
df2.to_csv("toner_data2.csv",index=False)

# 汇总所有页

In [7]:
df_combined_toner_1_7 = pd.concat([df,df2],axis=0, ignore_index=True)
df_combined_toner_1_7

product_name  \
0    Thayers Natural Remedies Witch Hazel Alcohol F...   
1    Thayers Natural Remedies 2% AHA Exfoliating To...   
2    The Ordinary Glycolic Acid 7% Toning Solution ...   
3    CeraVe Hydrating Toner for Face, Alcohol Free ...   
4    Neutrogena Alcohol-Free Gentle Daily Facial To...   
..                                                 ...   
147  Mydentity Guy Tang Demi-Permanent - My Dentity...   
148  Wella Color Charm Toner - Permanent Creme Tone...   
149  Alfaparf Milano Evolution of the Color Hair Dy...   
150  Schwarzkopf BlondMe Bond Enforcing Blonding To...   
151  Matrix SoColor Sync (6T Dark Titanium Blonde (...   

                        brand   current_price           unit_price  \
0    Thayers Natural Remedies  $4.49 - $12.39                  N/A   
1    Thayers Natural Remedies          $16.29                  N/A   
2                The Ordinary  $8.70 - $13.00                  N/A   
3                      CeraVe          $12.49  ($1.84/fluid ounce)   
4                  Neutrogena           $6.59  ($0.82/fluid ounce)   
..                        ...             ...                  ...   
147                #MYDENTITY          $17.99                  N/A   
148                     Wella          $12.99                  N/A   
149                  Alfaparf          $15.99                  N/A   
150               Schwarzkopf          $16.99                  N/A   
151                    Matrix          $19.99                  N/A   

                              star_rating review_counts  is_on_sale  \
0    4.7 out of 5 stars with 4108 ratings  4108 reviews           0   
1     4.8 out of 5 stars with 213 ratings   213 reviews           0   
2    4.4 out of 5 stars with 3342 ratings  3342 reviews           0   
3    4.7 out of 5 stars with 1535 ratings  1535 reviews           0   
4     4.2 out of 5 stars with 424 ratings   424 reviews           0   
..                                    ...           ...         ...   
147                                   N/A     0 reviews           0   
148                                   N/A     0 reviews           0   
149     2.3 out of 5 stars with 3 ratings     3 reviews           1   
150                                   N/A     0 reviews           0   
151                                   N/A     0 reviews           0   

    is_free_shipping  is_3day_delivery product_category  \
0                N/A                 1       Face Toner   
1                  0                 1       Face Toner   
2                N/A                 1       Face Toner   
3                  0                 1       Face Toner   
4                  0                 1       Face Toner   
..               ...               ...              ...   
147                0                 0       Face Toner   
148                0                 0       Face Toner   
149                0                 0       Face Toner   
150                0                 0       Face Toner   
151                0                 0       Face Toner   

                                          product_link  
0    https://www.target.com/p/thayers-natural-remed...  
1    https://www.target.com/p/thayers-natural-remed...  
2    https://www.target.com/p/the-ordinary-glycolic...  
3    https://www.target.com/p/cerave-hydrating-tone...  
4    https://www.target.com/p/neutrogena-alcohol-fr...  
..                                                 ...  
147  https://www.target.com/p/mydentity-guy-tang-de...  
148  https://www.target.com/p/wella-color-charm-ton...  
149  https://www.target.com/p/alfaparf-milano-evolu...  
150  https://www.target.com/p/schwarzkopf-blondme-b...  
151  https://www.target.com/p/matrix-socolor-sync-6...  

[152 rows x 11 columns]

In [8]:
df_combined_toner_1_7.to_csv("toner_1_7.csv",index=False)